In [ ]:
# check location of backend
import subprocess
import json
proc=subprocess.Popen('curl ipinfo.io', shell=True, stdout=subprocess.PIPE, )
ip_data = json.loads(proc.communicate()[0])
server_country = ip_data['country']
print(f"Server location:   {ip_data['city']} ({ip_data['region']}), {server_country}\n")

In [ ]:
project_id = 'YOUR_PROJECT_ID'
!gcloud config set project {project_id}

# connect to gs://
from google.colab import auth
auth.authenticate_user()

# Connect to Google Drive 
# The program code is assumed to be on Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Set environment variable so service accounts gets access to bucket (needed for gspath)
# (for more info see: https://cloud.google.com/docs/authentication/getting-started)
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/gdrive/JSON_WITH_SERVICE_ACCOUNT_PRIVATE_KEYS"

In [ ]:
### ======================== RUN PARAMETERS ======================= ###
###                                                                 ###
# dict with bucket-region pairs
# script will pick bucket in same region as backend to avoid expensive e-gress charges
# when training on TPUs YOUR_BUCKET_REGION should be US since all Colab TPUs are the US region
BUCKETS = {'gs://YOUR_BUCKET_NAME/': ['YOUR_BUCKET_REGION']}   

# Location and type of source files (on gs://...)
DATA_DIR = 'FILEPATH_OF_TFRECORD_FILES' # directory containing .tfrecords dataset (don't prefix with gs://YOUR_BUCKET_NAME)
BASE_DIR = 'FILEPATH_FOR_TRAINING'      # directory where training data will be stored (don't prefix with gs://YOUR_BUCKET_NAME)
TRAIN_SUB_DIR = ''                      # when empty, code will create a new subdirectory
INFER_DIR = './infer/'                  # sub-directory inside BASE_DIR for inference data
N_DISCR = 2
REPLICA_BATCH = 8
###                                                                 ###
### =============================================================== ###

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
import re

# select target bucket, based on country of backend (avoid e-gress!!!)
target_bucket = None
for bucket, country_lst in BUCKETS.items():
  if server_country in country_lst:
    target_bucket = bucket
    break
if target_bucket is None: 
  raise ValueError(f'No target-bucket found for {server_country}')
print(f"Target-bucket:     {target_bucket}")

# add target-bucket to directories
DATA_DIR = target_bucket + DATA_DIR
BASE_DIR = target_bucket + BASE_DIR

In [ ]:
# install modules used by the code
!pip install tensorboardx
!pip install soundfile
!pip install tensorflow_addons

In [ ]:
# install script to profile the TPU. This script is launched from the Python code of the model
!pip install --upgrade "cloud-tpu-profiler>=1.15.0rc1"

# install the tensorboard profile plugin (capture the data via capture-tpu-profile which is launched from the Python code!)
!pip install -U tensorboard_plugin_profile

# load tensorboard magic
%load_ext tensorboard

In [ ]:
# Make sure python finds the imports
import sys
sys.path.append('/content/gdrive/PATH_TO/audiocodec')
sys.path.append('/content/gdrive/PATH_TO/mp3net')

# local install of audiocodec (only needs to be executed once)
!pip install -e /content/gdrive/My\ Drive/program/MLTrance/audiocodec

In [ ]:
TRAIN_SUB_DIR = TRAIN_SUB_DIR if TRAIN_SUB_DIR != '' else None
NUM_REPLICA = 8
from argparse import Namespace
args = Namespace(mode='train',
                 training_base_dir=BASE_DIR, 
                 training_sub_dir=TRAIN_SUB_DIR,
                 infer_dir=INFER_DIR,
                 data_dir=DATA_DIR, 
                 n_discr=N_DISCR,
                 summary_freq=250,
                 runtime_tpu=True,
                 tpu_profiling=False,
                 runtime_eval_process=True, 
                 runtime_launch_tensorboard=False,
                 batch_size=NUM_REPLICA*REPLICA_BATCH, data_shuffle_buffer_size=NUM_REPLICA*REPLICA_BATCH*64, train_checkpoint_freq=200)
silent = True

import launcher 
setattr(args, "training_dir", launcher.get_training_dir(args, confirm=not silent and (args.mode=='train')))

SUMMARY_DIR = f"{args.training_dir}/summary/"
setattr(args, "summary_dir", SUMMARY_DIR)


In [ ]:
# Start a TPU
# note: all Google colab TPUs in 2019 were located in the US region (https://github.com/googlecolab/colabtools/issues/597)
if args.runtime_tpu:
  print("starting TPU...")
  tpu_strategy = launcher.setup_tpu()
else:  
  tpu_strategy = None
  print("GPU information:")
  !nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [ ]:
# run tensorboard (note: keeping tensorboard running implies class A and B operations and hence GSC costs)
print(f"Summary directory = {SUMMARY_DIR}")
%tensorboard --logdir=$SUMMARY_DIR

In [ ]:
# ===>>> RUN <<<===
import launcher
launcher.execute(args, tpu_strategy)